# Hands-on 6: Uso do exemplo rate-adaptation-distance.cc

## Objetivos:

 Este Hands-on descreve o funcionamento do exemplo rate-adaptation-distance.cc, que servirá de base para o desenvolvimento das análises da Fase 02 deste tutorial. Faça uma cópia do exemplo **rate-adapdation-distance.cc** (~/ns-allinone-3.29/ns-3.29/examples/wireless) e coloque na pasta ~/ns-allinone-3.29/ns-3.29/scratch.
 
 
## Versão desse tutorial

* ns-3.29.

 
## Uso do exemplo "rate-adaptation-distance.cc":

### Introdução

Em redes Wi-Fi não se tenta transmitir sempre a uma taxa de transmissão constante. O uso de algoritmos que adaptam a taxa de transmissão de acordo com as variações do canal é essencial para que se consiga a melhor transmissão possível. Mudanças no trafégo por usuário, no número de estações conectadas à rede ou distância entre receptor e transmissor são exemplos de características que influenciam diretamente no valor da melhor taxa de transmissão. Existem vários tipos de algoritmos, os implementados no ns-3 são: ArfWifiManager, onoeWifiManager, ConstantRateWifiManager, MinstrelWifiManager, IdealWifiManager, AarfWifiManager, AmrrWifiManager, CaraWifiManager, RraaWifiManager, AarfcdWifiManager, ParfWifiManager, AparfWifiManager.

#### Aarf (Adaptive Automatic Rate Fallback)

É um mecanismo simples, que considera apenas as perdas por desvanescimento do canal. Ele diminui a taxa de transmissão para a próxima taxa mais baixa disponível após 2 pacotes sucessivos perdidos. Após 10 transmissões de sucesso, ele aumenta a taxa para a próxima mais elevada disponível. o Décimo primeiro pacote é um pacote de sondagem, caso ele seja perdido, a taxa volta para a anterior. Em canais instáveis, esse algoritmo causa flutuações na rede, pois elevar a taxa pode levar a perdas e consequentes elevações seguidas de diminuição da taxa, resultando em uma instabilidade no valor da taxa de transmissão. 

#### Onoe 

Os primeiros pacotes são transmitidos a uma taxa inicial; após 1 segundo de transmissão as informações sob as transmissões são analisadas. Quando menos de 10% dos pacotes não precisaram ser reetransmitidos, o crédito é aumentado. Quando o crédito excede 10, a taxa de transmissão é aumentada e o crédito é zerado. Já quando a reetransmissão acontece para mais de 10% dos pacotes, o crédito é diminuido. Quando o cŕedito tem valor abaixo de zero, a taxa é reduzida e o crédito é zerado.

Esse algorítmo se caracteriza por ser um pouco mais completo, porém é lento, devido as análises a cada 1 segundo.

#### Minstrel

É um algoritmo mais avançado, que faz atualizações periódicas a cada 100 ms. 10% dos pacotes transmitidos são pacotes de sondagem enviados a taxas aleatórias com o intuito de obter informações sob o canal. Se um pacote é transmitido corretamente, não é necessário testar outras  taxas. Se a transmissão falha, o pacote é retransmitido utilizando outras taxas. Se o pacote é de sondagem, a sequência de tentativas é: taxa aleatória, taxa de melhor vazão útil, melhor probabilidade, taxa básica. Se o pacote é de carga útil, a sequência é: taxa de melhor vazão, próxima taxa de melhor vazão, melhor probabilidade, taxa básica. 

A fim de que em um canal com bom comportamento não exista perda de tempo com taxas baixas, a taxa aleatória é colocada em segundo lugar quando ela é menor que a melhor vazão útil. Assim o algoritmo sempre inicia com a maior taxa. Quando o canal não está bom, a transmissão irá falhar, e a taxa aleatória é colocada em tentativa. 

#### RRAA (Robust Rate Adaptation Algorithm)

Este algoritmo evita perdas de pacotes devido a colisão com um mecanismo de RTS adaptativo e é baseado na medição da perda de pacote em uma janela de transmissão. O algorimto inicia com a maior taxa de transmissão possível e cada taxa selecionada é utilizada durante toda a janela de transmissão. Ao fim da janela de transmissão, a taxa de perda de pacote é calculada e a taxa é atualizada na proxima janela. A taxa é reduzida para a próxima mais baixa quando a PLR é menor que a Pmtl (Limite Máximo de Perda Tolerável). A taxa é aumentada para a próxima superior quando a PLR é maior  que a *pori* (Limite Oportunístico de Aumento de Taxa). Se a PLR estiver entre os dois limites (*pori* e pmtl), a taxa é mantida, pois o canal nem está ruim para a taxa atual nem está preparado para um taxa maior. 

#### CARA (Strategy Collision-Aware Rate Adaptation)

Seu funcionamento é parecido com o aarf, porém ele usa RTS/CTS para reduzir colisões de pacotes.

 
  **Referências**
  1. http://blog.cerowrt.org/papers/minstrel-sigcomm-final.pdf
  2. https://www.cs.odu.edu/~cs752/papers/rate-002.pdf
  3. https://pdfs.semanticscholar.org/add6/2af7e541945d920cdab2aff0e3f1ec03f85d.pdf
  4. https://pdfs.semanticscholar.org/7ef0/88fd3192c78ed303a5a53db7447a09009383.pdf
  5. https://pdfs.semanticscholar.org/7ef0/88fd3192c78ed303a5a53db7447a09009383.pdf
  6. https://pdfs.semanticscholar.org/0536/387d23887d6f1c71e023e0421d94ce98a7c2.pdf
  

### Como funciona o exemplo"rate-adaptation-distance.cc ?"

Este exemplo cria uma infraestrutura simples de dois nós, na qual um AP gera tráafico UDP para uma STA. A STA é configurada para se mover em relação ao AP, gerando como saída o plot de *throughput vs. distance*.

No início, são adicionadas todas as bibliotecas necessárias para a execução do código.

![title](FIGS/1.png)


Após habilitação dos  componentes de registro é criada a classe **NodeStatistics** na qual são criados vários atributos públicos e privados. A definição de cada um dos atributos públicos é feita após a criação da classe e explicado posteriormente.

![title](FIGS/2.png)

Na imagem acima, a variável **m_bytesTotal** irá representar a quantidade total de bytes transmitidos a cada intervalo de tempo de análise. Neste código, os nós se deslocam *1 m* a cada *1 s*, logo o intervalo de tempo "steptime" é 1 segundo, e "stepsize" é 1 metro. **m_bytesTotal** poderá ser modificada através de um dos atributos públicos definidos a seguir.

A variável  **m_output** guarda os dados de posição *versus* throughput para plotagem do gráfico através do gnuplot. Ela poderá ser alterada através de um dos atributos públicos definidos a seguir.


#### Definição de cada atributo público:

Primeiro, é definido o construtor da classe, o qual inicializa a variável privada **m_bytesTotal** contendo o valor zero sempre que o objeto da classe é instanciado.  

![title](FIGS/3.png)
A função **RxCallback** pega o valor do tamanho do pacote passado como parâmetro, e adiciona ao valor já contido em **m_bytestotal**.

![title](FIGS/4.png)

A função **SetPosition** muda a posição do nó no modelo de mobilidade de acordo com a variavél **position** recebida como parâmetro.
![title](FIGS/5.png)

A função **GetPosition** retorna a posição atual do nó.

![title](FIGS/6.png)

O objetivo da função **AdvancePosition** é colocar os dados de posição e taxa de transmissão atual do nó na variável **m_output** e, em seguida, mandar o nó para sua próxima posição.

Primeiro ele usa a função já definida **GetPosition** para pegar a posição atual do nó. Na segunda linha é calculado o valor de taxa de transmissão em Mbps através do valor total de bytes transmitidos e o valor do intervalo de tempo correspondente. A quarta linha adiciona os valores encontrados de posição e a taxa em **m_output**, que será utilizada para plotar o gráfico com todas as posições *versus* taxa.

A quinta linha define qual será a próxima posição do nó (o nó está se deslocando apenas no eixo x, no valor de **StepSize**, que nesse caso será definido como 1 m). Em seguida a função **SetPosition** é utilizada para mandar o nó para essa nova posição definida. Finalmente, a última linha garante que a função **AdvancePosition** será chamada a cada valor de **StepTime**, calculando o valor de posição e taxa em todos os intervalos de tempo.

![title](FIGS/7.png)

Abaixo, se o logging está habilitado as mudanças no valor da taxa de transmissão poderão ser impressas. 

![title](FIGS/8.png)

Finalmente, a função **GetDataFile()** é definida, a qual retorna a variável **m_output**. 

![title](FIGS/9.png)

### Começando o código
O script começa definindo o *default* das suas principais variáveis, como qual padrão de rede Wi-Fi será utilizado, quais algoritmos de adaptação serão utilizados no AP e no STA, as posições iniciais dos nós e como eles se deslocam. Observe todas as variáveis. Em seguida é adicionado o código para que todas essas variáveis possam ser definidas através de linha de comando. Por último (na imagem abaixo), com o número de passos definido e o tempo de cada passo, é calculado o tempo total da simulação.

###### Sobre "ShortGuardInterval"
Uma das variáveis é **ShortGuardInterval**. *Short Guard Interval* é uma melhoria técnica que veio com o padrão de Wi-Fi 802.11 n. *Guard Interval* é o intervalo entre símbolos utilizado para evitar interferência intersimbólica. Normalmente o valor desse intervalo de guarda é 800 ns, com o "ShortGuardInterval" este intervalo passa a ser 400 ns. Com o *ShortGuardInterval*, há a diminuição de *overhead* e consequente aumento do *throughput*. Entretanto, em certas condições de canal, o *throughput* pode piorar quando este intervalo pequeno não é suficiente para evitar interferência intersimbólica. Esta opção só pode ser usada para os padrões 802.11 n/ac/ax. 

###### "Sobre RtsThreshold"
Uma das variáveis é o **RtsThreshold**. RTS/CTS podem ser utilizados em redes Wi-Fi para evitar colisões. Antes de enviar um pacote, uma sta deve enviar um RTS para o AP. **RtsThreshold** diz o tamanho máximo de bytes que uma STA pode enviar sem precisar enviar um novo RTS.

##### "Sobre BE_MaxampduSize"

MPDU é um dos tipos de agregação de quadros definidos a partir do 802.11 n. Transmitir um quadro em uma rede Wi-Fi possui *overhead* significativo. A agregação permite agregar quadros em um só, de forma a dimiuir o *overhead* total na rede. **BE_MaxampduSize** diz o tamanho máximo dessa agregação em bytes.

![title](FIGS/10.png)




Na figura abaixo os nós são criados, a camada física é configurada e, por último, os *containers* de dispositivos de rede são criados. A criação e instalação dos dispostivos nos nós ocorrerá apenas posteriormente. 

Observe que agora, além de configurar o canal da camada física, através do comando "WifiPhy.Set()" foi configurado se será utilizado ou não o *Short Guard Interval* de acordo com o valor *booleano* da variável **ShortGuardInterval**. Muitas outras variáveis podem ser alteradas através do comando **WifiPhy.Set()**. Consulte o [Doxygen](https://www.nsnam.org/docs/release/3.29/doxygen/classns3_1_1_yans_wifi_phy.html) do ns-3 para mais informações.

![title](FIGS/11.png)


O próximo passo é a configuração da camada MAC e criação e instalação dos dispositivos. Mas essa configuração será diferente para cada padrão. Aqui, após a criação de um objeto **WifiHelper** uma sequência de condições *if's* define o que acontece em cada padrão:

Para 802.11a/b/c: após a criação de um **WifiMacHelper**, o algoritmo de adaptação utilizado pelo objeto **WifiHelper** é definido com o valor de **rtsThreshold**. A configuração da camada MAC e instalação do dispositivo na STA é realizada como já aprendido anteriormente. No AP, tudo ocorre de forma semelhante:
 
![title](FIGS/12.png)
 
Para o 802.11 n: o mesmo ocorre, a diferença é que o valor **BE_MaxampduSize** também faz parte da configuração da camada MAC.
 
![title](FIGS/13.png)

Exatamente o mesmo procedimento ocorre para o 802.11 ac. Agora que todas as opções foram definidas, o dispositivo sta e Ap são adicionados aos *containers* de dispositivos Wi-Fi.

![title](FIGS/14.png)
 
 A linha abaixo sobrescreve o valor de *default* de *ChannelWidth* do padrão especificado para o valor contido em **ChWidth**.

![title](FIGS/16.png)
 
 Em seguida um modelo de mobilidade constante é definido tanto para STA como para o AP. Apesar do modelo de mobilidade constante, fazemos a STA se mover em relação ao AP através de método **AdvancePosition** da classe **NodeStatistics**
 
![title](FIGS/17.png)
 
 Um objeto da classe **NodeStatistics**, chamado **atpCounter** é criado, e **m_totalbytes** é inicializado com zero. Em seguida, o primeiro evento é agendado para ser executado durante (0.5 + *stepsTime*) segundos. O evento agendado é o método **AdvancePosition**, cujo retorno é guardado em **atpCounter** e seus argumentos são o nó STA, **stepsSize** e **stepsTime**. 
 
![title](FIGS/18.png)

O pŕoximo passo é instalar a pilha de protocolos de internet e definir os endereços dos dispositivos: 
    
![title](FIGS/20.png)

Abaixo, **PacketSinkHelper** ajuda a instalar um **PacketSinkApplication** em um conjunto de nós. Esta aplicação foi escrita para complementar a aplicação onoff, e se trata de uma aplicação simples apta a receber pacotes. Os parâmetros do *helper* são o endereço e porta do socket Rx e o protocolo de transporte a ser utilizado. A aplicação **PacketSink** possui uma fonte que rastreia quando um pacote é recebido e fornece acesso ao pacote recebido e endereço. Esta fonte será útil no código.

**OnOffHelper** ajuda a criar e instalar um **OnOffApplication** em um conjunto de nós. Os parâmetros são o protocolo usado para enviar o tráfego e o endereço para qual se deseja enviar. Uma aplicação *OnOff* simplesmente gera um tráfego para um único destino de acordo com um padrão *On/Off*: no estado *On* ele envia tráfego e no estado Off não. Os estados On e Off se alternam. o tráfego gerado no estado On é caracterizado pela taxa de dados e tamanho do pacote. No código abaixo, com o método **SetConstantRate** a taxa de dados é configurada para 420 mb/s e o tamanho do pacote é configurado para 1420 bytes. A aplicação irá começar em 0.5 segundos e finalizar de acordo com o valor de **simuTime**. A aplicação é instalada no AP, em seguida a aplicação **PacketSink** também é configurada para começar e finalizar nos mesmos tempos.

![title](FIGS/21.png)

Na imagem abaixo, o primeiro **Config::Connect** conecta a fonte de rastreamento ao método **RxCallback** do objeto **atpCounter** da classe **NodeStatistics**. Sempre que um pacote é recebido o método **RxCallback** é chamado e portanto o valor de **m_totalbytes** é atualizado, e com o método **advancePosition** (chamado a cada **stepsTime** durante a simulação) o valor de *throughput* é calculado.

O segundo **Config::Connect** conecta a fonte de rastreamento **RateChange** ao método **RateCallback**, o qual irá imprimir cada mudança na velocidade se o **logging** estiver habilitado. Finalmente a simulação é iniciada, e o simulador é configurado  para finalizar em **simuTime** segundos.

![title](FIGS/22.png)

Com a simulação finalizada, o arquivo de saida é criado, através de um objeto gnuplot todas as configurações do gráfico são definidas, e os dados de *throughput* *versus* distância são escritos no arquivo de saida através do método **GetDatafile**.

### Desafio 1
Gere o gráfico *throughput versus Distância* para uma STA se movendo a cada 2 metros a cada 2 segundos. Devem existir 100 pontos no gráfico. Altere o padrão para 802.11n, o algoritmo de adaptação para HTminstrel, e habilite o *ShortGuardInterval*.

Resposta: [aqui](CODES/rate-adaptation-distance.cc)

Para gerar o gráfico rode o comando no terminal dentro da pasta principal do ns-3: 
                            
                            gnuplot "nome-do-seu-arquivo".plt
Sua saída deverá ser semelhante a:
![title](FIGS/23.png)

Gráfico disponível: [aqui](CODES/throughput-MinstrelHt.eps)

Dados do gráfico *.plt* disponíveis: [aqui](CODES/throughput-minstrelHT.plt)